## Imports

In [25]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [26]:
from path_explain import utils
utils.set_up_environment(visible_devices='0')

In [27]:
import tensorflow as tf
import tensorflow_datasets
import numpy as np
import scipy
from transformers import *
from plot.text import text_plot

## Data and Model Loading

In [35]:
task = 'sst-2'
num_labels = len(glue_processors[task]().get_labels())
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

In [36]:
large_config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=num_labels)
large_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', config=large_config)

In [37]:
config = DistilBertConfig.from_pretrained('distilbert-base-uncased', num_labels=num_labels, dim=72)
model = TFDistilBertForSequenceClassification(config=config)

In [70]:
opt = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=13-8)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
model.compile(optimizer=opt, loss=loss, metrics=[metric])

In [77]:
data, info = tensorflow_datasets.load('glue/sst2', with_info=True)

INFO:absl:Overwrite dataset info from restored data version.
INFO:absl:Field info.description from disk and from code do not match. Keeping the one from code.
INFO:absl:Field info.location from disk and from code do not match. Keeping the one from code.
INFO:absl:Reusing dataset glue (/homes/gws/psturm/tensorflow_datasets/glue/sst2/0.0.2)
INFO:absl:Constructing tf.data.Dataset for split None, from /homes/gws/psturm/tensorflow_datasets/glue/sst2/0.0.2


In [78]:
train_dataset = glue_convert_examples_to_features(data['train'], tokenizer, max_length=128, task=task)
valid_dataset = glue_convert_examples_to_features(data['validation'], tokenizer, max_length=128, task=task)
valid_dataset = valid_dataset.batch(16)

In [92]:
_ = model.predict(valid_dataset)

array([[-0.02245433, -0.00332783],
       [-0.02258434, -0.00324947],
       [-0.02278818, -0.00320506],
       ...,
       [-0.02278283, -0.00315616],
       [-0.02254031, -0.00312885],
       [-0.02276037, -0.00321589]], dtype=float32)

In [97]:
model.summary()

Model: "tf_distil_bert_for_sequence_classification_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
distilbert (TFDistilBertMain multiple                  5035536   
_________________________________________________________________
pre_classifier (Dense)       multiple                  5256      
_________________________________________________________________
classifier (Dense)           multiple                  146       
_________________________________________________________________
dropout_121 (Dropout)        multiple                  0         
Total params: 5,040,938
Trainable params: 5,040,938
Non-trainable params: 0
_________________________________________________________________


In [135]:
weight_list = []

for i in range(len(large_model.layers[0].weights)):
    large_weight = large_model.layers[0].weights[i]
    slice_indices = []
    for s in large_weight.get_shape().as_list():
        if s == 768:
            slice_indices.append(slice(0, 72))
        else:
            slice_indices.append(slice(None))
    slice_indices = tuple(slice_indices)
    sliced_large_weight = large_weight[slice_indices]
    weight_list.append(sliced_large_weight)

model.layers[0].set_weights(weight_list)

In [1]:
for item in valid_dataset.take(1):
    print(item)

NameError: name 'valid_dataset' is not defined